In [1]:
import cv2
import numpy as np
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
from prettytable import PrettyTable
import csv

# Function to calculate the minimum distance to the nearest neighbor for each centroid
def calculate_min_distances(centroids):
    min_distances = np.zeros(len(centroids))
    for i, center in enumerate(centroids):
        distances_to_others = np.sqrt((centroids - center)**2).sum(axis=1)
        sorted_distances = np.sort(distances_to_others)
        min_distances[i] = sorted_distances[1] if len(sorted_distances) > 1 else 0
    return min_distances

# Load the original image
original_image_path = #enter image path here
original_img = cv2.imread(original_image_path, cv2.IMREAD_COLOR)

# Convert to grayscale and apply Gaussian blur
gray_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2GRAY)
blurred_img = cv2.GaussianBlur(gray_img, (9, 9), 0)

# Adaptive thresholding
thresh_img = cv2.adaptiveThreshold(blurred_img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                   cv2.THRESH_BINARY, 11, 2)

# Find contours and filter out small ones
contours, _ = cv2.findContours(thresh_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
filtered_contours = [c for c in contours if cv2.contourArea(c) > 1]

# Calculate centroids
centroids = np.array([cv2.moments(c) for c in filtered_contours])
centroid_points = np.array([(int(M['m10']/M['m00']), int(M['m01']/M['m00'])) 
                            if M['m00'] != 0 else (0, 0) for M in centroids])

# Save image after blurring and clustering
image_with_points = original_img.copy()
for point in centroid_points:
    cv2.circle(image_with_points, point, 3, (0, 255, 0), -1)
intermediate_output_path = 'C:/Users/tnerger/Desktop/Nerv/1.jpg'
cv2.imwrite(intermediate_output_path, image_with_points)

# Apply DBSCAN clustering to centroids to filter out outliers--------------------------------------------------------------
dbscan_eps = 27  # Adjust this value if needed
db = DBSCAN(eps=dbscan_eps, min_samples=11).fit(centroid_points)
labels = db.labels_
largest_cluster_label = max(set(labels), key=list(labels).count)

# Create a mask to draw the circles for the largest cluster
mask_with_circles = np.zeros(original_img.shape[:2], dtype="uint8")
centroids_without_outliers = centroid_points[labels == largest_cluster_label]

# Draw filled circles on the mask for each centroid in the largest cluster
for center in centroids_without_outliers:
    cv2.circle(mask_with_circles, center, int(dbscan_eps), (255, 255, 255), -1)

# Create an RGBA image from the original image
rgba_image = cv2.cvtColor(original_img, cv2.COLOR_BGR2BGRA)

# Set transparent background where the mask is black
rgba_image[mask_with_circles == 0] = [0, 0, 0, 0]

# Draw filled circles with red color and full opacity on the RGBA image
for center in centroids_without_outliers:
    cv2.circle(rgba_image, center, int(dbscan_eps), (0, 0, 255, 255), -1)

# Save the final image with a transparent background as a PNG
final_output_path_png = #enter path here
cv2.imwrite(final_output_path_png, rgba_image)

# Create and save image with remaining points after DBSCAN without previous points
remaining_points_image = original_img.copy()  # Use original image for drawing
remaining_points_image[:] = (0, 0, 0)  # Set image to black

for point in centroids_without_outliers:
    cv2.circle(remaining_points_image, point, 2, (0, 0, 255), -1)  # Draw red points
    cv2.circle(remaining_points_image, point, int(dbscan_eps), (0, 0, 255), 2)  # Draw red circles around the points

remaining_points_output_path = #enter path here
cv2.imwrite(remaining_points_output_path, remaining_points_image)

# Calculate the minimum distances
min_distances = calculate_min_distances(centroid_points)

# Filter out distances that are greater than 0
filtered_distances = [distance for distance in min_distances if distance > 0]

# Create a PrettyTable and a list for CSV output
table = PrettyTable()
table.field_names = ["Nummer", "X-Koordinate", "Y-Koordinate", "Minimale Distanz"]

csv_data = []

# Add each row to the table and CSV data, excluding zero distances
num = 1
for i, (center, distance) in enumerate(zip(centroids_without_outliers, min_distances)):
    if distance > 0:
        x, y = center
        table.add_row([num, x, y, distance])
        csv_data.append([num, x, y, distance])
        num += 1

# Print the table
print(table)

# Save the data to a CSV file
csv_output_path = #enter path here
with open(csv_output_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Nummer", "X-Koordinate", "Y-Koordinate", "Minimale Distanz"])
    writer.writerows(csv_data)

# Create a histogram of the distances with integer bins starting from 0
plt.figure(figsize=(8, 6))  # Reduce the width by 20%
bins = np.arange(0, max(filtered_distances) + 2) - 0.5  # Create bins for each integer distance
plt.hist(filtered_distances, bins=bins, edgecolor='black', color='red')
plt.title('Histogram of Minimum Distances')
plt.xlabel('Distance / Pixel')
plt.ylabel('Number of Points')
plt.xticks(np.arange(0, max(filtered_distances) + 1, 10))  # Set x-ticks for 0, 10, 20, ...
plt.grid(axis='y')  # Only horizontal grid lines

# Adjust the layout to minimize the distance between the y-axis and 0
plt.gca().set_xlim(left=-0.5)

plt.savefig(#enter path here)
plt.show()

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
